In [14]:
!nvidia-smi

Sun Aug 20 00:04:59 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A4000                On | 00000000:01:00.0 Off |                  Off |
| 41%   48C    P8               15W / 140W|   1557MiB / 16376MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [15]:
import os
HOME = os.getcwd()
print(HOME)

/home/manidhar/Desktop/AryanWork/yolov8-seg-training


In [16]:
from ultralytics import YOLO


MODEL = "Trained_Models/nano_segment_model.pt"
model = YOLO(MODEL) 

In [17]:

def predict_raised_axle(axle_data,threshhold):

    import numpy as np

    has_raised_axle = False
    slopes = []
    axle_count = len(axle_data)-1
    
    np.sort(axle_data)


    for i in range(0,axle_count):
        slopes.append(int(-1)*(axle_data[axle_count][0]-axle_data[i][0])/(axle_data[axle_count][1]-axle_data[i][1]))

    std_dev = np.std(slopes)

    if(std_dev>=threshhold):
        has_raised_axle = True

    return has_raised_axle


    

In [18]:
# # {0: 'A-10-S-TANDEM', 1: 'A-10-TRIDEM', 2: 'AXLE', 3: 'COMBINED', 4: 'TYPE-2', 5: 'TYPE-2-S2', 6: 'TYPE-3', 7: 'UC'}

# def contains_raised_axle(filepath_of_image):

#     result = model(filepath_of_image)
#     predicted_boxes = []
#     predicted_classes = []
#     axle_data = []

#     if len(result[0]) !=0:
#         predicted_boxes = result[0].boxes.xywh.cpu().numpy()
#         predicted_classes= result[0].boxes.cls.cpu().numpy()

#     for classes in range(len(predicted_classes)):
#         if(predicted_classes[classes] == 2):
#             axle_data.append([predicted_boxes[classes][0],predicted_boxes[classes][1]])

#     return predict_raised_axle(axle_data)



In [19]:
def get_axle_inside_bounding_box(bounding_box,axle_data): # both in [x,y,x,y] format

    axles_inside_bounding_box = []

    bounding_box_x_top = bounding_box[0]
    bounding_box_y_top = bounding_box[1]
    bounding_box_x_bot = bounding_box[2]
    bounding_box_y_bot = bounding_box[3] 


    for data in axle_data:
        x_top = int(data[0])
        y_top = int(data[1])

        if(bounding_box_x_top<=x_top and x_top<=bounding_box_x_bot and bounding_box_y_top<=y_top and y_top<=bounding_box_y_bot):
            axles_inside_bounding_box.append([(data[0]+data[2])/2,(data[1]+data[3])/2])
    
    return axles_inside_bounding_box

In [20]:
# {0: 'A-10-S-TANDEM', 1: 'A-10-TRIDEM', 2: 'AXLE', 3: 'COMBINED', 4: 'TYPE-2', 5: 'TYPE-2-S2', 6: 'TYPE-3', 7: 'UC'}

# modified

def contains_raised_axle(filepath_of_image,threshold):

    import numpy

    result = model(filepath_of_image)
    predicted_boxes = []
    predicted_classes = []
    axles = []

    if len(result[0]) !=0:
        predicted_boxes = result[0].boxes.xyxy.cpu().numpy()
        predicted_classes= result[0].boxes.cls.cpu().numpy()

    for classes in range(len(predicted_classes)):
        if(predicted_classes[classes]==2):
            axles.append(predicted_boxes[classes])

    for classes in range(len(predicted_classes)):
        if(predicted_classes[classes] == 3):
            axles_inside_bounding_box = get_axle_inside_bounding_box(predicted_boxes[classes],axles)

    return predict_raised_axle(axles_inside_bounding_box,threshold)



In [21]:
# A-13 Test Image 2023-06-03 04_55_51-554578__00000019.jpg

contains_raised_axle("MASTERDATASET/test/images/2023-06-03-04_55_51-554578__00000019_jpg.rf.cd8c762b38893e474fce4f7703690f54.jpg",0.3)

Ultralytics YOLOv8.0.20 🚀 Python-3.8.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA RTX A4000, 16109MiB)
YOLOv8n-seg summary (fused): 195 layers, 3259624 parameters, 0 gradients, 12.0 GFLOPs


False

In [22]:
# Type-3-S2 Test Image mainak-3_198_jpg.rf.9ab83c35318e7699f6fc0446239d395b.jpg

contains_raised_axle("MASTERDATASET/test/images/mainak-1_211_jpg.rf.37fd2f72e4cf2a088807ef796a649d3d.jpg",0.3)

True

In [23]:
# Type-3-S2 Test Image sravya-14_08_47-766729__00000014_jpg.rf.04ff4fd7d96a0584de87da05258f6b4c.jpg

contains_raised_axle("MASTERDATASET/test/images/sravya-14_08_47-766729__00000014_jpg.rf.b0429e0f472d85d152cb0a5888c30934.jpg",0.3)

False

In [24]:
results = model.predict("MASTERDATASET/test/images/2023-06-03-04_55_51-554578__00000019_jpg.rf.cd8c762b38893e474fce4f7703690f54.jpg", save=True, hide_labels=True,boxes=False)

Results saved to runs/segment/predict


In [25]:
results = model.predict("MASTERDATASET/test/images/sravya-14_08_47-766729__00000014_jpg.rf.b0429e0f472d85d152cb0a5888c30934.jpg", save=True, hide_labels=True,boxes=False)

Results saved to runs/segment/predict


In [26]:
results = model.predict("MASTERDATASET/test/images/mainak-3_198_jpg.rf.6d9bec74eb2c786bcb5fd2879ef90e64.jpg", save=True, hide_labels=True,boxes=False)

Results saved to runs/segment/predict
